# Explore trade-weighted dollar indices on the FRED

Compare other USD currency pairs with trade-weighted indices

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

fred<-Fred()

In [ ]:
cutoff <- Sys.Date() - 15
fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%dollar%' & FREQUENCY == 'D') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    filter(MAX_TS >= cutoff) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS) %>%
    print(n=Inf)

### compare USDINR and the EM trade-weighted index

In [ ]:
emId <- -2146970542 #DTWEXEMEGS
inrId <- -2147478748 #DEXINUS, from the example: http://pluto.studio/user/shyams80/notebooks/docs-R/Fred.ipynb

startDt <- as.Date("2006-01-02")
endDt <- as.Date("2019-06-20")

#get EM index data
emDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == emId & TIME_STAMP >= startDt & TIME_STAMP <= endDt) %>% 
    select(TIME_STAMP, VAL))
emDf$TIME_STAMP <- as.Date(emDf$TIME_STAMP)

fredDetailsEm <- data.frame(fred$Meta() %>% filter(SERIES_ID == emId))


#get USDINR data
usdInrDf <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == inrId & TIME_STAMP >= startDt & TIME_STAMP <= endDt) %>% 
    select(TIME_STAMP, VAL))
usdInrDf$TIME_STAMP <- as.Date(usdInrDf$TIME_STAMP)

fredDetailsUsdInr <- data.frame(fred$Meta() %>% filter(SERIES_ID == inrId))

#convert to xts
emXts <- xts(emDf$VAL, emDf$TIME_STAMP)
usdInrXts <- xts(usdInrDf$VAL, usdInrDf$TIME_STAMP)

In [ ]:
#end-of months don't match, so force them to a common date
monthlies <- merge(Common.NormalizeMonthlyDates(monthlyReturn(emXts)), 
                   Common.NormalizeMonthlyDates(monthlyReturn(usdInrXts)))
names(monthlies) <-c('EM', 'USDINR')

print(head(monthlies))

In [ ]:
Common.PlotCumReturns(monthlies, "Fred: DTWEXEMEGS vs. DEXINUS", "Trade Weighted U.S. Dollar Index: Emerging Markets Economies, Goods and Services vs. USDINR")

### add the thai bhat to the comparison

In [ ]:
fred$Meta() %>% inner_join(fred$TimeSeries(), by = 'SERIES_ID') %>%
    filter(NAME %like% '%exchange%' & NAME %like% '%thai%' & FREQUENCY == 'D') %>%
    group_by(SERIES_ID, TICKER, NAME) %>%
    summarize(MIN_TS = min(TIME_STAMP), MAX_TS = max(TIME_STAMP)) %>%
    filter(MAX_TS >= cutoff) %>%
    select(SERIES_ID, TICKER, NAME, MIN_TS, MAX_TS) %>%
    print(n=Inf)

In [ ]:
c1Id <- -2147478736 #DEXTHUS 

#get 2nd currency data
c1Df <- data.frame(fred$TimeSeries() %>% filter(SERIES_ID == c1Id & TIME_STAMP >= startDt & TIME_STAMP <= endDt) %>% 
    select(TIME_STAMP, VAL))
c1Df$TIME_STAMP <- as.Date(c1Df$TIME_STAMP)

fredDetailsC1 <- data.frame(fred$Meta() %>% filter(SERIES_ID == c1Id))

#convert to xts
c1Xts <- xts(c1Df$VAL, c1Df$TIME_STAMP)

monthlies <- merge(monthlies, Common.NormalizeMonthlyDates(monthlyReturn(c1Xts)))
names(monthlies) <-c('EM', 'USDINR', 'USDTHB')

print(head(monthlies))

In [ ]:
Common.PlotCumReturns(monthlies, "Fred: DTWEXEMEGS vs. DEXINUS vs. DEXTHUS", "Trade Weighted U.S. Dollar Index: Emerging Markets Economies, Goods and Services vs. USDINR vs. USDTHB")

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)